In [78]:
%matplotlib inline
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Data preparation
Reading forecasts and observations

In [79]:
# forecast dimensions
N = 281
T = 60
# base_dir = 'd:/GitHub/python-2-work/' # work
base_dir = 'd:/_SRC_/ensemble-forecasting/' # home
fc_hiromb = np.loadtxt(base_dir + 'data/ensemble-forecasts/2013102012-HIROMB-S1.txt')[:N, :T+1] - 37.356
fc_baltp = np.loadtxt(base_dir + 'data/ensemble-forecasts/2013102012-BALTP-90M-GFS-S1.txt')[:N, :T+1]
fc_bsm = np.loadtxt(base_dir + 'data/ensemble-forecasts/2013102012-BSM-WOWC-HIRLAM-S1.txt')[:N, :T+1]
m = np.loadtxt(base_dir + 'data/ensemble-forecasts/2013102012-M-GI_C1NB_C1FG_SHEP_restored.txt')
# preparing measurements forecast
fc_m = np.zeros((N, T+1))
for i in range(N):
    for j in range(T+1):
        fc_m[i, j] = m[i*6+j, 2]
fc_hiromb.shape, fc_baltp.shape, fc_bsm.shape, fc_m.shape

((281, 61), (281, 61), (281, 61), (281, 61))

Selecting peaks

In [80]:
def get_peak_index(fc, guard_level, k):
    mask = fc > guard_level
    r_mask = np.concatenate(([False,], mask[:-1]))
    l_mask = np.concatenate((mask[1:], [False,]))
    idx = np.array([np.where(mask & ~r_mask)[0], np.where(mask & ~l_mask)[0]])    
    res_idx = np.empty((0, 4), int)
    for i in range(idx.shape[1]):
        res_idx = np.vstack((res_idx,
                             [k, idx[0, i], np.argmax(fc[idx[0, i]:idx[1, i] + 1]) + idx[0, i], idx[1, i]]))
    return res_idx
res_idx = np.empty((0, 4), int)
for i in range(N):
    res_idx = np.vstack((res_idx, get_peak_index(fc_baltp[i], 120, i)))
res_idx

array([[ 26,  54,  56,  58],
       [ 27,  48,  51,  53],
       [ 28,  42,  43,  45],
       [ 30,  32,  34,  35],
       [ 31,  26,  28,  30],
       [ 32,  21,  23,  26],
       [ 35,   4,   5,   5],
       [ 36,   0,   0,   1],
       [ 77,  39,  42,  44],
       [ 78,  34,  36,  39],
       [ 79,  29,  30,  31],
       [ 80,  23,  24,  24],
       [ 82,  12,  12,  12],
       [ 83,   5,   6,   7],
       [101,  58,  60,  60],
       [102,  54,  59,  60],
       [103,  48,  53,  56],
       [104,  43,  47,  50],
       [105,  36,  41,  44],
       [106,  30,  36,  40],
       [107,  23,  30,  34],
       [108,  18,  25,  29],
       [109,  13,  19,  22],
       [110,  10,  13,  15],
       [111,   4,   7,   8],
       [112,   0,   1,   3],
       [146,  58,  59,  60],
       [147,  53,  53,  53],
       [148,  47,  47,  48],
       [151,  28,  30,  31],
       [161,  47,  47,  47],
       [162,  41,  42,  43],
       [178,  48,  50,  51],
       [204,  60,  60,  60],
       [205,  